In [2]:
# all our imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from pickle import dump

In [3]:
def extract_features(directory):
    model = tf.keras.applications.VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    print(model.summary())
    features = dict()
    for name in listdir(directory):
        filename = directory + '/' + name
        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    image_id = name.split('.')[0]
    features[image_id] = feature
    return features

directory = 'instagram_data/img'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
dump(features, open('features.pkl', 'wb'))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-02-21 15:54:20.429300: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-21 15:54:20.429341: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

2024-02-21 15:56:50.487838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Extracted Features: 1


In [4]:
print(len(features['insta8959']))

1


In [40]:
import pandas as pd
def load_doc(filename):
    df = pd.read_csv(filename)
    # Remove  na values
    df = df.dropna()
    # Drop the first column
    df = df.drop(df.columns[0], axis=1)
    # Rename Image File column to image_id
    df = df.rename(columns={"Image File": "image_id"})
    # In the image_id column, remove the file extension
    df['image_id'] = df['image_id'].str.replace('img/', '')
    # Turn every letter lowercase
    df['Caption'] = df['Caption'].str.lower()

    # Turn the dataframe into a dictionary with the image_id as the key and the caption as the value
    captions = dict()
    for i in range(len(df)):
        image_id = df.iloc[i]['image_id']
        caption = df.iloc[i]['Caption']
        captions[image_id] = caption

    return captions
    

filename = 'instagram_data/captions_csv.csv'
descriptions = load_doc(filename)
print(descriptions)

{'insta2': 'bye ', 'insta3': 'ok, a few more... sorry i just had so much fun that day ', 'insta4': 'this was one of my favorite shoots i’ve ever done! thank you -a seriously badass woman, for shooting my cover! and thank you to everyone who asked a question for the interview 🤍 ', 'insta5': 'wrapped round my finger like a ring ', 'insta6': 'yesterday was so much fun!! has been such a huge figure in my life. i love her humor, her grace and her heart! check out the interview where i’m gushing over her the whole time! ', 'insta7': 'well.. ', 'insta8': 'it’s officially out! i was a bit embarrassed asking so often for you to stream or buy my album. it felt inauthentic. thank you so much for making something so personal to me be a moment i’ll never forget. all i truly desire is for you all to enjoy the music and spread the love 🦋🌈💫 ', 'insta9': 'i hope you guys are liking the album! i just want to thank you guys for supporting me and this body of work. it means the world to me.. you have no i

In [66]:
def to_vocabulary(descriptions):
    # build a list of all description strings
    all_desc = set()
    # Split the descriptions into words and add them to the set
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key].split()]
    return all_desc
 
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print(vocabulary)
print(len(vocabulary))

{'grandpas', "people's", '-they', 'racked', 'rock!', 'diary', '🌗', 'saving', 'cordon', 'pic.', 'bathroom,', 'go)', 'lit', 'fluff', 'hurt.', 'chat!', 'cultural', 'dude,', 'bound!', 'siblings', '👿', 'pesky', 'timeless.', 'initials!', 'hesitate', 'immersed', 'kind', 'each.', 'himself!', 'seeeee...fifth', 'headlining', 'deserved', 'picked', 'miss.', '☕️', 'youungggg~', 'iheart', 'butttt', 'momma,', 'lipstick', 'mthe', 'edgiest', 'auction', 'take-', '🐆', 'coma', 'supermarket', 'philly,', 'needed,', 'cosmo’s', 'beecroft', 'city.', 'self.', 'kenzo', 'slit', 'poured', 'follow!', 'warms', 'triple', 'cutie!', 'coffe', 'shaving', 'changing,', 'firey', 'alexander', 'la', 'sandcastlekings', '✌️🇺🇸', 'foundation', 'kimoji', 'sconces', 'kansas', 'hates', 'acoustic', '♥️✨', 'ordinary', 'once', 'kids?', 'jenner?', 'views.”', 'but!', 'see-through', 'paul’s', 'kalfanzo', 'errands', 'glosses', 'jimmy', "chilli's", '🔌', 'treatments.', 'newsstands', 'anyways...', 'rocky', '👱❤️', 'alert!!!!', '02.03', 'addict